In [1]:
import pygame
from pygame.locals import *

class Button(pygame.sprite.Sprite):
    def __init__(self, image, pos, text_input, font, base_color, hovering_color):
        super().__init__()
        self.image = image
        self.x_pos = pos[0]
        self.y_pos = pos[1]
        self.base_color, self.hovering_color = base_color, hovering_color
        self.text_input = text_input
        self.font = font
        self.text = self.font.render(self.text_input, True, self.base_color)
        if self.image is None:
            self.image = self.text
        self.rect = self.image.get_rect(center=(self.x_pos, self.y_pos))
        self.text_rect = self.text.get_rect(center=(self.x_pos, self.y_pos))

    def update(self, screen):
        if self.image is not None:
            screen.blit(self.image, self.rect)
        screen.blit(self.text, self.text_rect)

    def update_position(self, pos):
        self.rect.topleft = pos

    def checkForInput(self, position):
        if position[0] in range(self.rect.left, self.rect.right) and position[1] in range(self.rect.top, self.rect.bottom):
            return True
        return False

    def changeColor(self, position):
        if position[0] in range(self.rect.left, self.rect.right) and position[1] in range(self.rect.top, self.rect.bottom):
            self.text = self.font.render(self.text_input, True, self.hovering_color)
        else:
            self.text = self.font.render(self.text_input, True, self.base_color)

    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.checkForInput(pygame.mouse.get_pos()):
                # Do something when the button is clicked
                pass

    def handle_hover(self):
        if self.checkForInput(pygame.mouse.get_pos()):
            self.text = self.font.render(self.text_input, True, self.hovering_color)
        else:
            self.text = self.font.render(self.text_input, True, self.base_color)


pygame 2.5.1 (SDL 2.28.2, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import numpy as np
import pygame

ROW_COUNT = 15
COL_COUNT = 15

# define screen size
BLOCKSIZE = 30  # Adjusted for 960x540 resolution
PADDING_RIGHT = 150  # Adjusted for 960x540 resolution
RADIUS = 12  # Adjusted for 960x540 resolution

# colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BROWN = (205, 128, 0)

# create a board array
def create_board(row, col):
    board = np.zeros((row, col))
    return board

# draw a board in pygame window
def draw_board(screen, board_size):
    S_WIDTH = board_size * BLOCKSIZE
    S_HEIGHT = board_size * BLOCKSIZE
    for x in range(0, S_WIDTH, BLOCKSIZE):
        for y in range(0, S_HEIGHT, BLOCKSIZE):
            rect = pygame.Rect(x, y, BLOCKSIZE, BLOCKSIZE)
            pygame.draw.rect(screen, BROWN, rect)

    # draw inner grid lines
    # draw vertical lines
    for x in range(BLOCKSIZE // 2, S_WIDTH - BLOCKSIZE // 2 + BLOCKSIZE, BLOCKSIZE):
        line_start = (x, BLOCKSIZE // 2)
        line_end = (x, S_HEIGHT - BLOCKSIZE // 2)
        pygame.draw.line(screen, BLACK, line_start, line_end, 2)

    # draw horizontal lines
    for y in range(BLOCKSIZE // 2, S_HEIGHT - BLOCKSIZE // 2 + BLOCKSIZE, BLOCKSIZE):
        line_start = (BLOCKSIZE // 2, y)
        line_end = (S_WIDTH - BLOCKSIZE // 2, y)
        pygame.draw.line(screen, BLACK, line_start, line_end, 2)
    pygame.display.update()

# drop a piece
def drop_piece(board, row, col, piece):
    board[row][col] = piece

# draw a piece on the board
def draw_piece(screen, board, board_size):
    # draw game pieces at the mouse location
    for x in range(board_size):
        for y in range(board_size):
            circle_pos = (x * BLOCKSIZE + BLOCKSIZE // 2, y * BLOCKSIZE + BLOCKSIZE // 2)
            if board[y][x] == 1:
                pygame.draw.circle(screen, BLACK, circle_pos, RADIUS)
            elif board[y][x] == -1:
                pygame.draw.circle(screen, WHITE, circle_pos, RADIUS)
    pygame.display.update()

# check if it is a valid location
def is_valid_loc(board, row, col):
    return board[row][col] == 0

# victory decision
def who_wins(board, piece, board_size):
    # check for horizontal win
    for c in range(board_size - 4):
        for r in range(board_size):
            if board[r][c] == piece and board[r][c + 1] == piece and board[r][c + 2] == piece and board[r][c + 3] == piece \
                    and board[r][c + 4] == piece:
                return True

    # check for vertical win
    for c in range(board_size):
        for r in range(board_size - 4):
            if board[r][c] == piece and board[r + 1][c] == piece and board[r + 2][c] == piece and board[r + 3][c] == piece \
                    and board[r + 4][c] == piece:
                return True

    # check for positively sloped diagonal win
    for c in range(board_size - 4):
        for r in range(4, board_size):
            if board[r][c] == piece and board[r - 1][c + 1] == piece and board[r - 2][c + 2] == piece and board[r - 3][c + 3] == piece \
                    and board[r - 4][c + 4] == piece:
                return True

    # check for negatively sloped diagonal win
    for c in range(board_size - 4):
        for r in range(board_size - 4):
            if board[r][c] == piece and board[r + 1][c + 1] == piece and board[r + 2][c + 2] == piece and board[r + 3][c + 3] == piece \
                    and board[r + 4][c + 4] == piece:
                return True


In [3]:
import pygame,os,time
import sys
import pygame_gui
from Button import Button
from pygame.locals import *
from tkinter import *
from state import State
from title import Title
import Start
from testsetting import get_font
class Game:
    def __init__(self):
        pygame.init()
        self.GAME_W, self.GAME_H = 540,360
        self.game_canvas = pygame.Surface((self.GAME_W,self.GAME_H))
        self.screen = pygame.display.set_mode((960, 720), pygame.RESIZABLE)
        pygame.display.set_caption("Your Game Title")
        self.manager = pygame_gui.UIManager(self.screen.get_size())
        self.clock = pygame.time.Clock()
        self.bg = pygame.Surface((self.screen.get_width(), self.screen.get_height()))
        self.bg.fill((0, 0, 0))
        self.play_button = Button(image=None, pos=(640 * 3 // 4, 250 * 3 // 4),
                                text_input="PLAY", font=self.get_font(
                                    int(75 * 3 // 4 * self.screen.get_width() / 960)),
                                base_color="#d7fcd4", hovering_color="White")

        self.options_button = Button(image=None, pos=(640 * 3 // 4, 400 * 3 // 4),
                                    text_input="OPTIONS", font=self.get_font(
                                        int(75 * 3 // 4 * self.screen.get_width() / 960)),
                                    base_color="#d7fcd4", hovering_color="White")

        self.quit_button = Button(image=None, pos=(640 * 3 // 4, 550 * 3 // 4),
                                text_input="QUIT", font=self.get_font(
                                    int(75 * 3 // 4 * self.screen.get_width() / 960)),
                                base_color="#d7fcd4", hovering_color="White")
        self.resume_button = Button(image=None, pos=(640 * 3 // 4, 550 * 3 // 4),
                                text_input="RESUME", font=self.get_font(
                                    int(75 * 3 // 4 * self.screen.get_width() / 960)),
                                base_color="#d7fcd4", hovering_color="White"
        )
        self.ingame_options_button = Button(image=None, pos=(640 * 3 // 4, 400 * 3 // 4),
                                    text_input="OPTIONS", font=self.get_font(
                                        int(75 * 3 // 4 * self.screen.get_width() / 960)),
                                    base_color="#d7fcd4", hovering_color="White")
        
        self.ingame_quit_button = Button(image=None, pos=(640 * 3 // 4, 550 * 3 // 4),
                                text_input="QUIT", font=self.get_font(
                                    int(75 * 3 // 4 * self.screen.get_width() / 960)),
                                base_color="#d7fcd4", hovering_color="White")
        self.in_game_menu_buttons = [self.resume_button, self.ingame_options_button, self.ingame_quit_button]

        self.load_states()

    def get_font(self, size):
        return pygame.font.Font("font.ttf", size)

    def update_button_text_size(self, button, base_size):
            current_size = int(base_size * self.screen.get_width() / 960)
            button.font = self.get_font(current_size)

    def update(self):
            self.state_stack[-1].update(self.dt,self.actions)
    def render(self):
        self.state_stack[-1].render|self.game_canvas
        self.screen.blit(pygame.transform.scale(self.game_canvas, (self.SCREEN_WIDTH, self.SCREEN_HEIGHT), (0,0)))
        pygame.display.flip()
    def get_dt(self):
        now = time.time()
        self.dt= now - self.prev_time
        self.prev_time = now
    def load_states(self):
        self.title_screen = Title(self)
        self.state_stack.append(self.title_screen)

    def run(self):
        while True:
            
            mouse_pos = pygame.mouse.get_pos()

            self.screen.blit(self.bg, (0, 0))

            # Handle events, update game logic, and draw UI
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == VIDEORESIZE:
                    self.screen = pygame.display.set_mode((event.w, event.h), pygame.RESIZABLE)
                    self.update_button_text_size(self.play_button, 75)
                    self.update_button_text_size(self.options_button, 75)
                    self.update_button_text_size(self.quit_button, 75)

            pygame.display.update()
            self.clock.tick(60)

    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.VIDEORESIZE:
                self.handle_resize(event)
            # Add other event handling as needed

    def play(self):
        while True:
            mouse_pos = pygame.mouse.get_pos()

            self.screen.blit(self.bg, (0, 0))

            human_button = Button(image=None, pos=(640 * 3 // 4, 400 * 3 // 4),
                                  text_input="vs Human", font=self.get_font(60 * 3 // 4),
                                  base_color="White", hovering_color="Pink")

            ai_button = Button(image=None, pos=(640 * 3 // 4, 300 * 3 // 4),
                               text_input="vs AI", font=self.get_font(60 * 3 // 4),
                               base_color="White", hovering_color="Pink")

            back_button = Button(image=None, pos=(640 * 3 // 4, 600 * 3 // 4),
                                text_input="BACK", font=self.get_font(60 * 3 // 4),
                                base_color="White", hovering_color="Pink")

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.MOUSEBUTTONDOWN:
                    if human_button.checkForInput(mouse_pos):
                        Start.main()
                    if ai_button.checkForInput(mouse_pos):
                        pass
                    if back_button.checkForInput(mouse_pos):
                        self.main_menu()

            for button in [human_button, ai_button, back_button]:
                button.changeColor(mouse_pos)
                button.update(self.screen)

            pygame.display.update()

    def options(self):
        while True:
            mouse_pos = pygame.mouse.get_pos()

            self.screen.blit(self.bg, (0, 0))

                # Board size button
            size_button = Button(image=None, pos=(640*3//4, 400*3//4), 
                            text_input="Board Size", font= get_font(60*3//4), 
                            base_color="White", hovering_color="Pink")
            
            # Win condition button
            win_button = Button(image=None, pos=(640*3//4, 300*3//4), 
                            text_input="Win Condition", font=get_font(60*3//4), 
                            base_color="White", hovering_color="Pink")

            # Back button                   
            back_button = Button(image=None, pos=(640*3//4, 600*3//4), 
                                text_input="BACK", font=get_font(60*3//4), 
                                base_color="White", hovering_color="Pink")

            for button in [size_button, win_button, back_button]:
                    button.changeColor(mouse_pos)
                    button.update(self.screen)

            for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()
                    if event.type == pygame.MOUSEBUTTONDOWN:
                        if size_button.checkForInput(mouse_pos):
                            self.enter_boardsize()
                        if win_button.checkForInput(mouse_pos):
                            self.enter_wincondition()
                        if back_button.checkForInput(mouse_pos):
                            self.main_menu()

            pygame.display.update()

    def enter_boardsize(self):
        
        while True:
            mouse_pos = pygame.mouse.get_pos()

            self.screen.blit(self.bg, (0, 0))

            self.ui_refresh_rate = self.clock.tick(60 * 3 // 4) / 1000

            enter_text = self.get_font(45 * 3 // 4).render("Enter the board size", True, "White")
            enter_rect = enter_text.get_rect(center=(640 * 3 // 4, 260 * 3 // 4))

            self.screen.blit(enter_text, enter_rect)

            back_button = Button(image=None, pos=(640 * 3 // 4, 600 * 3 // 4),
                                text_input="BACK", font=self.get_font(60 * 3 // 4),
                                base_color="White", hovering_color="Pink")

            back_button.changeColor(mouse_pos)
            back_button.update(self.screen)
    def enter_wincondition(self):
        while True:
            win_mouse_pos = pygame.mouse.get_pos()

            self.screen.blit(self.bg, (0, 0))

            enter_text = self.get_font(45).render("Enter the win condition", True, "White")
            enter_rect = enter_text.get_rect(center=(640 * 3 // 4, 260 * 3 // 4))
            self.screen.blit(enter_text, enter_rect)

            back_button = Button(image=None, pos=(640 * 3 // 4, 600 * 3 // 4),
                                text_input="BACK", font=self.get_font(60),
                                base_color="White", hovering_color="Pink")

            back_button.changeColor(win_mouse_pos)
            back_button.update(self.screen)  # Move this line outside of the event loop

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.MOUSEBUTTONDOWN:
                    if back_button.checkForInput(win_mouse_pos):
                        self.options()

            if (event.type == pygame_gui.UI_TEXT_ENTRY_FINISHED and
                    event.ui_object_id == '#main_text_entry'):
                Start.board_size = int(event.text)
                self.options()
            self.manager.process_events(event)
            ui_refresh_rate = self.clock.tick(60 * 3 // 4) / 1000
            self.manager.update(ui_refresh_rate)

            self.manager.draw_ui(self.screen)

            pygame.display.update()


    def resizable_loop(self):
        full_screen = False
        while True:
            self.screen.fill((0, 0, 50))
            pygame.draw.rect(self.screen, (255, 0, 0),
                            pygame.Rect(self.screen.get_width() - 5 - (self.screen.get_width() / 5), 50,
                                        self.screen.get_width() / 5, 50))
            for event in pygame.event.get():
                if event.type == QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == VIDEORESIZE:
                    if not full_screen:
                        self.screen = pygame.display.set_mode((event.w, event.h), pygame.RESIZABLE)
                if event.type == KEYDOWN:
                    if event.key == K_ESCAPE:
                        pygame.quit()
                        sys.exit()

            pygame.display.update()
            self.clock.tick(60)
    def in_game_menu(self):
        while True:
            mouse_pos = pygame.mouse.get_pos()

            self.screen.blit(self.bg, (0, 0))

            for button in self.in_game_menu_buttons:
                button.changeColor(mouse_pos)
                button.update(self.game_canvas)

            # Draw the in-game menu on the screen
            self.screen.blit(pygame.transform.scale(self.game_canvas, (self.SCREEN_WIDTH, self.SCREEN_HEIGHT)), (0, 0))

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == VIDEORESIZE:
                    self.handle_resize(event)

                # Handle events for in-game menu buttons
                for button in self.in_game_menu_buttons:
                    button.handle_event(event)

            pygame.display.update()
            self.clock.tick(60)

    def handle_in_game_menu(self):
        self.in_game_menu()  # Call this method when you want to show the in-game menu
def in_game(self):
    while True:
        mouse_pos = pygame.mouse.get_pos()

        self.screen.blit(self.bg, (0, 0))

        # Check for events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == VIDEORESIZE:
                self.handle_resize(event)

        # Draw your in-game content

        # Check if the user wants to open the in-game menu
        keys = pygame.key.get_pressed()
        if keys[K_ESCAPE]:
            self.handle_in_game_menu()  # Open the in-game menu

        pygame.display.update()
        self.clock.tick(60)

# Add handle_event and handle_hover methods to the Button class
def handle_event(self, event):
    if event.type == pygame.MOUSEBUTTONDOWN:
        if self.checkForInput(pygame.mouse.get_pos()):
            # Do something when the button is clicked
            pass

def handle_hover(self):
    if self.checkForInput(pygame.mouse.get_pos()):
        self.text = self.font.render(self.text_input, True, self.hovering_color)
    else:
        self.text = self.font.render(self.text_input, True, self.base_color)
class InGameMenu:
    def __init__(self, game):
        self.game = game
        self.buttons = [
            Button("Resume", (640 * 3 // 4, 250 * 3 // 4), self.resume_game),
            Button("Options", (640 * 3 // 4, 400 * 3 // 4), self.show_options),
            Button("Quit", (640 * 3 // 4, 550 * 3 // 4), self.quit_game)
        ]

    def resume_game(self):
        self.game.switch_state("in_game")  # Switch back to the in-game state

    def show_options(self):
        self.game.switch_state("options_menu")  # Switch to the options menu

    def quit_game(self):
        pygame.quit()
        sys.exit()

    def handle_events(self, events):
        for event in events:
            if event.type == pygame.MOUSEBUTTONDOWN:
                for button in self.buttons:
                    if button.checkForInput(pygame.mouse.get_pos()):
                        button.execute_action()

    def update(self):
        pass  # Add any update logic if needed

    def render(self, screen):
        screen.blit(self.game.bg, (0, 0))
        for button in self.buttons:
            button.changeColor(pygame.mouse.get_pos())
            button.update(screen)


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

SystemExit: 

C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
import pygame
import sys
from Button import Button
from Game import Game
from pygame.locals import *
from tkinter import *
def main_menu(self):
        while True:
            self.screen.blit(self.bg, (0, 0))

            menu_mouse_pos = pygame.mouse.get_pos()
            self.handle_events()    

            # Adjust the size of the menu text based on the screen width
            menu_text = self.get_font(int(100 * 3 // 4 * self.screen.get_width() / 960)).render(
                "MAIN MENU", True, "#b68f40")
            menu_rect = menu_text.get_rect(center=(self.screen.get_width() // 4, self.screen.get_height() // 4))

            # Calculate button positions based on the screen size
            button_y = self.screen.get_height() // 2

            play_button = Button(image=pygame.image.load("Play Rect.png"),
                                pos=(self.screen.get_width() // 2, button_y),
                                text_input="PLAY",
                                font=self.get_font(int(75 * 3 // 4 * self.screen.get_width() / 960)),
                                base_color="#d7fcd4", hovering_color="White")

            options_button = Button(image=pygame.image.load("Options Rect.png"),
                                    pos=(self.screen.get_width() // 2, button_y + 150),
                                    text_input="OPTIONS",
                                    font=self.get_font(int(75 * 3 // 4 * self.screen.get_width() / 960)),
                                    base_color="#d7fcd4", hovering_color="White")

            quit_button = Button(image=pygame.image.load("Quit Rect.png"),
                                pos=(self.screen.get_width() // 2, button_y ),
                                text_input="QUIT",
                                font=self.get_font(int(75 * 3 // 4 * self.screen.get_width() / 960)),
                                base_color="#d7fcd4", hovering_color="White")
        
            # Update button positions when the screen is resized
            play_button.update_position((self.screen.get_width() // 2, button_y))
            options_button.update_position((self.screen.get_width() // 2, button_y + 300))
            quit_button.update_position((self.screen.get_width() // 2, button_y + 300))

            self.screen.blit(menu_text, menu_rect)

            for button in [play_button, options_button, quit_button]:
                button.changeColor(menu_mouse_pos)
                button.update(self.screen)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.MOUSEBUTTONDOWN:
                    if play_button.checkForInput(menu_mouse_pos):
                        self.play()
                    if options_button.checkForInput(menu_mouse_pos):
                        self.options()
                    if quit_button.checkForInput(menu_mouse_pos):
                        pygame.quit()
                        sys.exit()

            self.resizable_loop()  # Call resizable_loop in the event loop
            pygame.display.update()

# Create an instance of the Game class and run the game
game_instance = Game()
game_instance.main_menu()

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
[[ 0.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
[[ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
[[-1.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
[[-1.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1. -1.]]
[[-1.  0.  1.]
 [ 0.  1.  0.]
 [-1.  1. -1.]]
[[-1.  0.  1.]
 [ 1.  1.  0.]
 [-1.  1. -1.]]
[[-1.  0.  1.]
 [ 1.  1. -1.]
 [-1.  1. -1.]]
[[-1.  1.  1.]
 [ 1.  1. -1.]
 [-1.  1. -1.]]
[[-1.  1.  1.]
 [ 1.  1. -1.]
 [-1.  1. -1.]]


SystemExit: 